## Adding a lightning logging
* If you log your loss with the prog_bar turned on in training or validation step with self.log, then you will see the loss in the progress bar.
* Here is a snippet code:
```
class LitModel(pl.LightningModule):  
   ... 
   def validation_step(self, batch, batch_idx):
        ...

        # logs metrics for each training_step,
        self.log("val_loss", loss, prog_bar=True)
        return loss
```

In [3]:
import os
import torch
import pytorch_lightning as pl
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
from torch.nn import functional as F

from torchmetrics import Accuracy

class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
        

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        #logits = self(x)
        #loss = F.nll_loss(logits, y)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        #preds = torch.argmax(logits, dim=1)
        preds = torch.argmax(y_hat, dim=1)
        self.val_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        #self.log("val_acc", self.val_accuracy, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

BATCH_SIZE = 128 if torch.cuda.is_available() else 32
#train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))

train_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4, 
    batch_size = BATCH_SIZE
)

val_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4,
    batch_size = BATCH_SIZE
)

#mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
#mnist_train, mnist_val = random_split(mnist_full, [55000, 5000])


trainer = pl.Trainer(
    accelerator="auto",
    max_epochs = 5
)
model = LitModel()
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | l1           | Linear             | 7.9 K 
1 | val_accuracy | MulticlassAccuracy | 0     
----------------------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
